In [1]:
import urllib.request
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
try:
    import ipywidgets as widgets
    hasWidgets = True
except ImportError:
    hasWidgets = False
import geopandas as gpd
import glob
import json
import requests
import shapely


In [2]:
#https://data.cityofnewyork.us/dataset/New-York-City-Population-By-Census-Tracts/kc6e-jm93
#https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku

In [3]:
gdf=gpd.GeoDataFrame.from_file('geo_export_63fb6c72-b31e-4794-b2b1-b6554cc9bff2.shp')
gdf['boro_ct201']=gdf[['boro_ct201']].astype(np.int64)
#gdf.head(1)
gdf1 = gdf[['boro_ct201','ntaname','geometry']]
gdf1.head(1)
gdf.shape

(2166, 12)

In [4]:
df = pd.read_csv('New_York_City_Population_By_Census_Tracts.csv')
df['boro_ct201'] = df['DCP Borough Code']* 1000000 + df['Census Tract']
#df.groupby('boro_ct201').count().sort_values('Year',ascending=True)
df = df[(df['Year']==2010)]
df1 = df[['boro_ct201','Population']]
df1.head(1)
df1.shape

(2168, 2)

In [5]:
gdf2 = gdf1.merge(df1, on = 'boro_ct201')
gdf2['centroid'] = gdf2.geometry.centroid
gdf2.head(1)
gdf3 = gdf2[['boro_ct201','geometry','centroid']]
gdf3.rename(columns={'geometry':'poly','centroid':'geometry'},inplace=True)
gdf4 = gdf3[['boro_ct201','geometry']]
gdf4.tail(1)
gdf4.crs
#type(gdf4)

/nfshome/ku373/.conda/envs/env_ver3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


{'init': 'epsg:4326'}

There are some duplicated stop_id, but don't care for now.

In [6]:
bus = pd.read_csv('bus_stop_output_df (1).csv')
bus1 = bus[['stop_id','stop_lat','stop_lon','stop_name']]
#bus.groupby('stop_id').count().sort_values('lonlat')
#bus.head(1)
crs = {'init': 'epsg:4326'}

bus1['lonlat']=list(zip(bus1.stop_lon,bus1.stop_lat))
bus1['geometry'] = bus1[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
bus2 =gpd.GeoDataFrame(bus1, crs=crs, geometry=bus1['geometry'])
type(bus2)

/nfshome/ku373/.conda/envs/env_ver3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/nfshome/ku373/.conda/envs/env_ver3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


geopandas.geodataframe.GeoDataFrame

In [10]:
def min_distance(point, lines):
    return lines.distance(point).min()

gdf4['min_dist_bus'] = gdf4.geometry.apply(min_distance, args=(bus2,))

In [12]:
gdf4.head()

,boro_ct201,geometry,min_dist_to_lines
0,5000900,POINT (-74.08285639238751 40.64220415963202),0.002005
1,1009800,POINT (-73.9677747247446 40.75578800678499),0.001369
2,1010000,POINT (-73.97123160585845 40.75806566605351),0.000104
3,1010200,POINT (-73.97444936939443 40.75942376908788),0.000341
4,1010400,POINT (-73.97767288514945 40.76077898596836),0.001612


Subway

In [15]:
sub = pd.read_csv('shapes.txt')
sub.head(1)

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon
0,1..N03R,42159,40.702068,-74.013664


In [16]:
sub['lonlat']=list(zip(sub.shape_pt_lon,sub.shape_pt_lat))
sub['geometry'] = sub[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
sub1 =gpd.GeoDataFrame(sub, crs=crs, geometry=sub['geometry'])
type(sub1)

geopandas.geodataframe.GeoDataFrame

In [18]:
def min_distance(point, lines):
    return lines.distance(point).min()

gdf4['min_dist_sub'] = gdf4.geometry.apply(min_distance, args=(sub1,))

In [23]:
#gdf4.rename(columns={'min_dist_sub':'min_dist_sub1'},inplace=True)
gdf4.head(1)

,boro_ct201,geometry,min_dist_sub1,min_dist_sub1
0,5000900,POINT (-74.08285639238751 40.64220415963202),0.002005,0.05785


In [51]:
gdf5 = gdf4.drop(columns=['geometry'],axis=1)

In [52]:
gdf5 = gdf2.merge(gdf5, on='boro_ct201')
type(gdf5)

geopandas.geodataframe.GeoDataFrame

In [54]:
#gdf5.iloc[:,7]
gdf5.head()

,boro_ct201,ntaname,geometry,Population,centroid,min_dist_sub1,min_dist_sub1
0,5000900,West New Brighton-New Brighton-St. George,POLYGON ((-74.07920577013245 40.64343078374567...,2112,POINT (-74.08285639238751 40.64220415963202),0.002005,0.057850
1,1009800,Turtle Bay-East Midtown,POLYGON ((-73.96432543478758 40.75638153099091...,7316,POINT (-73.9677747247446 40.75578800678499),0.001369,0.002180
2,1010000,Turtle Bay-East Midtown,POLYGON ((-73.96802436915851 40.75957814005282...,1992,POINT (-73.97123160585845 40.75806566605351),0.000104,0.001180
3,1010200,Midtown-Midtown South,POLYGON ((-73.97124277307127 40.76093641847906...,230,POINT (-73.97444936939443 40.75942376908788),0.000341,0.001074
4,1010400,Midtown-Midtown South,POLYGON ((-73.97445730550224 40.76229308352487...,966,POINT (-73.97767288514945 40.76077898596836),0.001612,0.000593


In [67]:
import scipy.stats
gdf5['z_sub'] = scipy.stats.zscore(gdf5.iloc[:,6],axis=0)
gdf5['z_bus'] = scipy.stats.zscore(gdf5.iloc[:,5],axis=0)
gdf5['z_pop'] = scipy.stats.zscore(gdf5.iloc[:,3],axis=0)
gdf5['z_popdens'] = scipy.stats.zscore(gdf5.iloc[:,3]/gdf6.geometry.area,axis=0)

In [68]:
gdf6 = gdf5.drop(columns=['centroid'],axis=1)
gdf6.to_file('test.shp')

In [69]:
type(gdf2)

geopandas.geodataframe.GeoDataFrame

In [70]:
gdf6.Population / gdf6.geometry.area

0       8.551879e+07
1       3.874372e+08
2       1.080431e+08
3       1.247423e+07
4       5.228944e+07
5       6.245627e+06
6       2.274656e+08
7       2.206965e+08
8       4.093449e+08
9       5.165912e+08
10      4.151848e+08
11      2.783126e+08
12      1.851430e+08
13      1.700144e+08
14      7.245774e+08
15      5.200659e+08
16      4.222087e+08
17      4.495565e+08
18      4.057509e+08
19      4.759428e+08
20      1.042953e+08
21      2.084453e+08
22      1.532488e+08
23      1.743778e+08
24      1.984472e+08
25      1.854003e+08
26      3.634056e+08
27      2.953375e+08
28      3.393756e+08
29      2.798075e+08
            ...     
2136    1.999105e+08
2137    1.770339e+08
2138    5.047574e+07
2139    1.119286e+08
2140    1.503191e+08
2141    1.849118e+08
2142    1.289561e+07
2143    1.106284e+08
2144    1.078212e+07
2145    7.746034e+07
2146    3.630650e+07
2147    2.834322e+08
2148    1.107412e+08
2149    6.096618e+06
2150    4.430127e+08
2151    1.887096e+07
2152    3.543